In [6]:
import cv2
from ultralytics import YOLO
import numpy as np

In [15]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLO pose model
model = YOLO(r'models\train2-20250307T113013Z-001\train2\weights\best.pt')

# Load the image
image_path = r"receipt_Sample.jpg"
image = cv2.imread(image_path)

# Perform inference
results = model(image)

# Eğer model herhangi bir kutu algıladıysa işle
if results[0].boxes is not None and results[0].keypoints is not None:
    boxes = results[0].boxes.xyxy.cpu().numpy()  # (Nesne Sayısı, 4) → [x1, y1, x2, y2]
    box_confidences = results[0].boxes.conf.cpu().numpy()  # (Nesne Sayısı,) → Kutuların güven skorları
    keypoints = results[0].keypoints.xy.cpu().numpy()  # (Nesne Sayısı, Keypoint Sayısı, 2)
    keypoint_confidences = results[0].keypoints.conf.cpu().numpy()  # (Nesne Sayısı, Keypoint Sayısı)

    # En yüksek güven skoruna sahip kutuyu seç
    best_box_index = box_confidences.argmax()
    best_box = boxes[best_box_index]  # [x1, y1, x2, y2]

    # En iyi kutuya ait keypointleri al
    best_keypoints = keypoints[best_box_index]  # (Keypoint Sayısı, 2)
    best_keypoint_confidences = keypoint_confidences[best_box_index]  # (Keypoint Sayısı,)

    # Kutuyu çiz
    x1, y1, x2, y2 = map(int, best_box)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)  # Yeşil kutu

    # En iyi 4 keypoint'i seç ve çiz
    top_4_indices = best_keypoint_confidences.argsort()[-4:]  # En yüksek 4 güvene sahip noktayı al
    for idx in top_4_indices:
        x, y = map(int, best_keypoints[idx])
        cv2.circle(image, (x, y), 8, (0, 0, 255), -1)  # Kırmızı noktalar

# Pencere boyutunu ayarla
max_size = 800  # Maksimum genişlik/yükseklik
height, width = image.shape[:2]

if max(height, width) > max_size:
    scale = max_size / max(height, width)
    new_size = (int(width * scale), int(height * scale))
    image = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)

# Show the image with the best box and keypoints
cv2.imshow('Best Box & Keypoints', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 2016x928 5 receipts, 677.8ms
Speed: 40.0ms preprocess, 677.8ms inference, 2.9ms postprocess per image at shape (1, 3, 2016, 928)


# segmentasn 


In [8]:
import os,cv2
import numpy as np
from ultralytics import YOLO
# path="C:\\Users\\denem\\Desktop\\roboflow_156_foot\\train\\images\\"
# path="C:\\Users\\denem\\Desktop\\realtest\\"
path=r"images/"
# path="C:\\Users\\denem\\Desktop\\realtest_net\\"
files=os.listdir(path)
files[0],len(files)

('fis_100_jpg.rf.4479da7899ddf514a2cfba9d87b7f832.jpg', 156)

In [ ]:


# YOLO segmentasyon modelini yükle
model = YOLO(r'models\train-seg\train\weights\best.pt')



0: 2016x1152 1 Receipt, 967.8ms
Speed: 34.3ms preprocess, 967.8ms inference, 10.5ms postprocess per image at shape (1, 3, 2016, 1152)


In [ ]:

# Resmi yükle
image_path = path+files[7]
image = cv2.imread(image_path)

# Model ile tahmin yap
results = model(image)

# Segmentasyon maskesi için bir boş görüntü oluştur
mask = np.zeros(image.shape[:2], dtype=np.uint8)

# Eğer segmentasyon sonuçları varsa devam et
if results[0].masks is not None:
    masks = results[0].masks.xy  # Maskelerin kontur noktaları (x, y) formatında

    # Her bir maskeyi çiz
    for mask_points in masks:
        points = np.array(mask_points, dtype=np.int32)
        cv2.fillPoly(mask, [points], 255)  # Maskeyi beyaz ile doldur

    # Orijinal görüntü ile maskeyi birleştirerek göster
    segmented_image = cv2.bitwise_and(image, image, mask=mask)

    # Maske görselleştirmesi için renkli hale getir (mavi overlay ekledik)
    overlay = cv2.merge([mask, np.zeros_like(mask), np.zeros_like(mask)])
    output = cv2.addWeighted(image, 0.7, overlay, 0.3, 0)

else:
    output = image.copy()  # Eğer mask bulunamazsa orijinal görüntüyü göster

# ** Pencere Boyutunu Ayarla **
max_size = 800  # Maksimum genişlik/yükseklik
height, width = output.shape[:2]

if max(height, width) > max_size:
    scale = max_size / max(height, width)
    new_size = (int(width * scale), int(height * scale))
    output = cv2.resize(output, new_size, interpolation=cv2.INTER_AREA)

# Sonucu göster
cv2.imshow('Segmented Image', output)
cv2.waitKey(0)
cv2.destroyAllWindows()
